## Python Library for Neural Network and Deep Learning

Most of the neural network libraries in Python requires [Theano](https://github.com/Theano/Theano) as a core library. Here is few easy-to-install and easy-to-implement libraries for neural network:

- [Lasagne](https://github.com/Lasagne/Lasagne) (has very good documentation [here]( http://lasagne.readthedocs.org/en/latest/index.html))

Here are wrapper libraries for `Lasagne`:
- [scikit-neuralnetwork](https://github.com/aigamedev/scikit-neuralnetwork)
- [nolearn](https://github.com/dnouri/nolearn) > we'll try `nolearn` today, since it's the simplest one

I would suggest starting with **Lasagne** then go for others. Below is how to define each network layers for Lasagne (example from Github)

```python
import lasagne
import theano
import theano.tensor as T

# create Theano variables for input and target minibatch
input_var = T.tensor4('X')
target_var = T.ivector('y')

# create a small convolutional neural network
from lasagne.nonlinearities import leaky_rectify, softmax
network = lasagne.layers.InputLayer((None, 3, 32, 32), input_var)
network = lasagne.layers.Conv2DLayer(network, 64, (3, 3),
                                     nonlinearity=leaky_rectify)
network = lasagne.layers.Conv2DLayer(network, 32, (3, 3),
                                     nonlinearity=leaky_rectify)
network = lasagne.layers.Pool2DLayer(network, (3, 3), stride=2, mode='max')
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    128, nonlinearity=leaky_rectify,
                                    W=lasagne.init.Orthogonal())
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    10, nonlinearity=softmax)
```

## Facebook Human or Robots

### Predict if an online bid is made by a machine or a human

feature was created using this script: https://github.com/small-yellow-duck/facebook_auction by [small-yellow-duck](https://github.com/small-yellow-duck). You can simply run the code to get feature set or just ask me anytime. In the given feature set, features with `outcome` column > 0 are training set and < 0 are testing set.

We divide `X, y` into `X_train, y_train` then use 80% from training set to train classifier and 20% of training set for cross validation

Submissions are judged on area under the ROC curve.

In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split

In [2]:
facebook_feature = pd.read_csv('facebook_feature.csv') # feature

In [3]:
cutoff = facebook_feature[facebook_feature.outcome>=0].shape[0]
y =  facebook_feature['outcome'].values
X = preprocessing.normalize(facebook_feature.drop('outcome', 1).values, axis=0)

In [4]:
y_train = y[0:cutoff]
X_train = X[0:cutoff, :]

In [7]:
clf = RandomForestClassifier(n_estimators=800, max_depth=None, 
                             min_samples_leaf=1, random_state=0, 
                             criterion='entropy')
X_train_valid, X_valid, y_train_valid, y_valid = train_test_split(X_train, y_train, test_size=0.2)
clf.fit(X_train_valid, y_train_valid)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [8]:
y_hat = clf.predict_proba(X_valid)[:,1] # predicted 
print 'ROC score = ', roc_auc_score(y_valid, y_hat)

ROC score =  0.967940125112


Here, we will just simply try building Neural network classifier for the same problem using `nolearn` (wrapper for `Lasagne`)

http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/

In [9]:
from nolearn.lasagne import NeuralNet
from lasagne import layers
from lasagne.updates import nesterov_momentum

In [10]:
net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 408),  # 96x96 input pixels per batch
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # output layer uses identity function
    output_num_units=1, 

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.4,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=0,
    )

In [11]:
net1.fit(X_train_valid, y_train_valid)

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x109e67c50>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x109e67bd0>,
     custom_score=None, hidden_num_units=100, input_shape=(None, 408),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=400, more_params={},
     objective=<function objective at 0x109e645f0>,
     objective_loss_function=<function squared_error at 0x116d6c938>,
     on_batch_finished=[], on_epoch_finished=[], on_training_finished=[],
     on_training_started=[], output_nonlinearity=None, output_num_units=1,
     regression=True,
     train_split=<nolearn.lasagne.base.TrainSplit object at 0x109e67c90>,
     update=<function nesterov_momentum at 0x116d77050>,
     update_learning_rate=0.01, update_momentum=0.4,
     use_label_encoder=Fa

In [12]:
y_hat_nn = net1.predict_proba(X_valid) # predict probability output

In [13]:
print 'ROC score = ', roc_auc_score(y_valid, y_hat_nn)

ROC score =  0.813114387846
